In [1]:
from ngsolve import *
import netgen.geom2d as geom2d
from ngsolve.webgui import Draw
from netgen.occ import *

In [2]:
#genero la malla en el [0,1]x[0,1]

geo = geom2d.SplineGeometry()
p1 = geo.AppendPoint (0,0)
p2 = geo.AppendPoint (1,0)
p3 = geo.AppendPoint (1,1)
p4 = geo.AppendPoint (0,1)


geo.Append (["line", p1, p2],bc = "wall_1")
geo.Append (["line", p2, p3],bc = "wall_2")
geo.Append (["line", p3, p4],bc = "wall_3")
geo.Append (["line", p4, p1],bc = "wall_4")

mesh = Mesh(geo.GenerateMesh (maxh=0.05))
#mesh.SplitElements_Alfeld()
mesh.GetBoundaries()
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [3]:
V = VectorH1(mesh, order=2, dirichlet="wall_1|wall_2|wall_3|wall_3")
Q = L2(mesh, order=1)
X = V*Q

In [4]:
(u,p),(v,q) = X.TnT()

stokes = InnerProduct(Grad(u), Grad(v))*dx + div(u)*q*dx + div(v)*p*dx
a = BilinearForm(stokes).Assemble()

In [5]:
gf = GridFunction(X)
uh, ph = gf.components

g0 = CoefficientFunction((0,0))
g1 = CoefficientFunction((1,0))
uh.Set(g1, definedon=mesh.Boundaries("wall_3"))
uh.Set(g0, definedon=mesh.Boundaries("wall_1|wall_2|wall_4"))

In [7]:
res = -a.mat * gf.vec
inv = a.mat.Inverse(freedofs=X.FreeDofs(),inverse="pardiso")
gf.vec.data += inv * res
Draw(ph, mesh)

c:\Users\mauri\anaconda3\Lib\site-packages\netgen\webgui.py:17: RuntimeWarning: overflow encountered in cast
  values = np.array(data.flatten(), dtype=dtype)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene